In [58]:
import pandas as pd
import numpy as np 
from collections import Counter
from imblearn.over_sampling import SMOTE
from matplotlib import pyplot
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
def load_dataset(path,missing_headers=False):
    if missing_headers:
        data = pd.read_csv(path, header = None )
    else:
        data = pd.read_csv(path ,header = 0 )
    
#     global rows,columns
#     rows,columns = data.shape
    return data
def handling_missingData(data):
    data = data.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    have_nan =data.isin(['?']).sum() > 0
    data.replace('?', np.nan , inplace=True)
    lastindex = len(data.columns) -1
    X,y = data.drop(lastindex,axis =1),data[lastindex]
    categoryCols = X.select_dtypes(include=["object"]).columns
    numericCols = X.select_dtypes(include=["int64","float64"]).columns
    for column in categoryCols:
        if column in have_nan :
            data[column] = data[column].fillna(data[column].mode()[0])
    for column in numericCols:
        if column in have_nan :
            data[column] = data[column].fillna(data[column].mean())
    return data 
def Remove_outliers(data):
    lastindex = len(data.columns) -1
    numerixs = [i for i in range(len(data.iloc[0])) if type(data.iloc[0,i]) != str]

    mean = data.describe().iloc[1, :]
#     mean la vi tri index 1 trong discribe , lay toan bo mean cua cac cot cua data 
    std = data.describe().iloc[2, :]
#     std la vi tri index 2 trong discribe ,...
    
    for(numerix , mean, std) in zip(numerixs, mean, std):
        data = data[data[numerix] < 3*std + mean]
        
    return data 
def labelEncoder(data):
    for col in data.columns:
        if data[col].dtypes == 'object':
            le = LabelEncoder()
            data[col] = le.fit_transform(data[col].astype(str))
    return data 

    
def BalancingData(data):
    target = data.values[:,-1]
    inputData = data.values[:,:-1]
    counter = Counter(target)
    print(counter)
    
    for k,v in counter.items():
        per = v/len(target)*100
        print("Class%s,Counter%d,Percentage%3f%%",(k,v,per))
    
    overSample = SMOTE()
    A,b = overSample.fit_resample(inputData,target)
    
    counter = Counter(b)
    print( counter)
    return data,A,b
def standardizeFeatures(data):
    mmscaler = MinMaxScaler()
    data.columns =["age","workclass","fnlwgt","education","education-num","marital-status","occupation","relationship","race","sex","capital-gain","capital-loss","hours-per-week","native-country","Salary"]
    cols = ["age","fnlwgt","education-num","capital-gain","capital-loss","hours-per-week"]
    data[cols] = mmscaler.fit_transform(data[cols])
    data = pd.DataFrame(data[cols])
    return data
def evaluate_model(X, y, model):

    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

    scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
    return scores    
    
data = load_dataset("adult-all.csv" , missing_headers= True)
data.head()
# handling_missingData(data)
# Remove_outliers(data)
# standardizeFeatures(data)
# labelEncoder(data)
# BalancingData(data)
# target = data.values[:,-1]
# inputData = data.values[:,:-1]
# model = SVC(gamma='scale')
# scores = evaluate_model(inputData,target,model )
# print('Mean Accuracy: %.3f (%.3f)'% (np.mean(scores), np.std(scores)))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [59]:
handling_missingData(data)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [60]:
Remove_outliers(data)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [61]:
standardizeFeatures(data)

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
0,0.301370,0.044302,0.800000,0.021740,0.0,0.397959
1,0.452055,0.048238,0.800000,0.000000,0.0,0.122449
2,0.287671,0.138113,0.533333,0.000000,0.0,0.397959
3,0.493151,0.151068,0.400000,0.000000,0.0,0.397959
4,0.150685,0.221488,0.800000,0.000000,0.0,0.397959
...,...,...,...,...,...,...
32556,0.136986,0.166404,0.733333,0.000000,0.0,0.377551
32557,0.315068,0.096500,0.533333,0.000000,0.0,0.397959
32558,0.561644,0.094827,0.533333,0.000000,0.0,0.397959
32559,0.068493,0.128499,0.533333,0.000000,0.0,0.193878


In [62]:
labelEncoder(data)


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,Salary
0,0.301370,7,0.044302,9,0.800000,4,1,1,4,1,0.021740,0.0,0.397959,39,0
1,0.452055,6,0.048238,9,0.800000,2,4,0,4,1,0.000000,0.0,0.122449,39,0
2,0.287671,4,0.138113,11,0.533333,0,6,1,4,1,0.000000,0.0,0.397959,39,0
3,0.493151,4,0.151068,1,0.400000,2,6,0,2,1,0.000000,0.0,0.397959,39,0
4,0.150685,4,0.221488,9,0.800000,2,10,5,2,0,0.000000,0.0,0.397959,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,0.136986,4,0.166404,7,0.733333,2,13,5,4,0,0.000000,0.0,0.377551,39,0
32557,0.315068,4,0.096500,11,0.533333,2,7,0,4,1,0.000000,0.0,0.397959,39,1
32558,0.561644,4,0.094827,11,0.533333,6,1,4,4,0,0.000000,0.0,0.397959,39,0
32559,0.068493,4,0.128499,11,0.533333,4,1,3,4,1,0.000000,0.0,0.193878,39,0


In [63]:
BalancingData(data)


Counter({0.0: 24720, 1.0: 7841})
Class%s,Counter%d,Percentage%3f%% (0.0, 24720, 75.91904425539757)
Class%s,Counter%d,Percentage%3f%% (1.0, 7841, 24.080955744602438)
Counter({0.0: 24720, 1.0: 24720})


(            age  workclass    fnlwgt  education  education-num  \
 0      0.301370          7  0.044302          9       0.800000   
 1      0.452055          6  0.048238          9       0.800000   
 2      0.287671          4  0.138113         11       0.533333   
 3      0.493151          4  0.151068          1       0.400000   
 4      0.150685          4  0.221488          9       0.800000   
 ...         ...        ...       ...        ...            ...   
 32556  0.136986          4  0.166404          7       0.733333   
 32557  0.315068          4  0.096500         11       0.533333   
 32558  0.561644          4  0.094827         11       0.533333   
 32559  0.068493          4  0.128499         11       0.533333   
 32560  0.479452          5  0.187203         11       0.533333   
 
        marital-status  occupation  relationship  race  sex  capital-gain  \
 0                   4           1             1     4    1      0.021740   
 1                   2           4      

In [64]:
target = data.values[:,-1]
inputData = data.values[:,:-1]
model = DecisionTreeClassifier()

scores = evaluate_model(inputData,target,model )
print('Mean Accuracy: %.3f (%.3f)'% (np.mean(scores), np.std(scores)))

Mean Accuracy: 0.809 (0.005)


In [73]:
model.fit(inputData, target)
test=  pd.read_csv("adult-test.csv")
test.head()

,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0.1,40,United-States,<=50K.
0,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
1,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
2,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
3,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K.
4,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K.


In [81]:
test = load_dataset("adult-test.csv" , missing_headers= True)
test.head()
handling_missingData(test)
standardizeFeatures(test)
labelEncoder(test)

datatest = test.values[:,:-1]
predictions = model.predict(datatest)
output = pd.DataFrame({  'income': predictions })
output.to_csv('results.csv', index=None)